In [ ]:
#use from pip install -r requirements.txt

import requests
from bs4 import BeautifulSoup
import sqlite3
from io import BytesIO
from PIL import Image

# download an image and convert it to a BLOB
def image_to_blob(url):
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    blob = BytesIO()
    img.save(blob, format='PNG')  # Convert the image to PNG format for simplicity
    return blob.getvalue()

# Connect to SQLite database
conn = sqlite3.connect('recipes.db')
c = conn.cursor()

# Create a table if it doesn't exist
c.execute('''CREATE TABLE IF NOT EXISTS recipes (
                id INTEGER PRIMARY KEY,
                title TEXT,
                ingredients TEXT,
                instructions TEXT,
                categories TEXT,
                details TEXT,
                img1 BLOB
            )''')
conn.commit()

# Iterate over URLs from 1 to 600
for i in range(1, 600):
    url = f'https://inarodaram.ir/view/{i}'
    response = requests.get(url)
    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')

       # Extract recipe information
        title = soup.find('h1', class_='fw-bold text-center text-danger mb-3 mt-2')
        title = title.text.strip() if title else 'Title not found'
        instructions_heading = soup.find('h3', text="دستور پخت")
        if instructions_heading:
            instructions_list = [li.get_text(strip=True) for li in instructions_heading.find_next('ul').find_all('li')]
            instructions = ', '.join(instructions_list)

        ingredients_heading = soup.find('h3', text="موارد مورد نیاز")
        ingredients_list = [li.text.strip() for li in ingredients_heading.find_next('ul').find_all('li')] if ingredients_heading else []
        ingredients = ', '.join(ingredients_list) if ingredients_list else 'Ingredients not found'

        categories_heading = soup.find('span', class_='fw-bold', text="دسته بندی:")
        categories = ', '.join([a.text.strip() for a in soup.select('span.fw-bold + a')]) if categories_heading else 'Categories not found'

        details = soup.find('div', class_='mt-1').text.strip() if soup.find('div', class_='mt-1') else 'Details not found'

        first_image_url = soup.find('div', class_='food-image-large')['style'].split("url('")[1].split("')")[0]

# Convert the first image to a BLOB
        image_blob = image_to_blob("https://inarodaram.ir" + first_image_url)    

        # Insert recipe into the database including images
        c.execute("INSERT INTO recipes (id, title, ingredients, instructions, categories, details, img1) VALUES (?, ?, ?, ?, ?, ?, ?)",
                  (i, title, ingredients, instructions, categories, details, image_blob))
        conn.commit()
    else:
        print(f"Failed to fetch data from URL: {url}")

# Close the database connection
conn.close()
